![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [48]:
import pandas as pd
import numpy as np

all_data = pd.read_csv('bank_marketing.csv') 
all_data.sample(n = 5)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
36314,36314,28,self-employed,single,university.degree,no,yes,yes,cellular,jun,8,187,2,999,0,nonexistent,-2.9,92.963,-40.8,1.266,5076.2,no
11699,11699,31,blue-collar,single,basic.4y,no,no,no,telephone,jun,7,110,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
36348,36348,67,admin.,married,basic.4y,unknown,yes,no,cellular,jun,6,177,2,999,1,failure,-2.9,92.963,-40.8,1.262,5076.2,no
1798,1798,53,self-employed,married,university.degree,no,no,no,telephone,may,23,404,2,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
38442,38442,35,unemployed,married,university.degree,no,no,no,cellular,oct,28,344,1,6,1,success,-3.4,92.431,-26.9,0.733,5017.5,yes


In [49]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

## step 1 Client: splitting, Cleaning and Saving the data 

In [50]:
# all col we need
client = all_data[["client_id","age","job","marital","education","credit_default","housing","loan"]]

# Rename columns 
client.rename(columns = {'client_id':'id'}, inplace=True)

# Cleaning colmns 
client['education'] = client['education'].str.replace( "." , "_" ) 
client['education'] = client['education'].replace("unknown",np.NaN)

# Save the DataFrames to csv file without an index
client.to_csv("client.csv", index=False) 


## step 2 Campaing: splitting, Cleaning and Saving the data 

In [51]:
# whitout col campaign_id and last_contact_date
campaign = all_data[["client_id","campaign","month","day","duration","pdays","previous","poutcome","y"]] 

# Rename columns 
campaign.rename(columns = {'duration' : 'contact_duration',
                           'previous' : 'previous_campaign_contacts',
                           'y' : 'campaign_outcome',
                           'poutcome' : 'previous_outcome',
                           'campaign' : 'number_contacts'
                          }, inplace=True)

# Cleaning colmns 
campaign['campaign_outcome'] = campaign['campaign_outcome'].map({"no":0, "yes": 1})
campaign['previous_outcome'] = campaign['previous_outcome']\
                                .map({"failure":0, "success":1,"nonexistent": np.NaN}) 

# Creating new column campaign_id
campaign["campaign_id"] = 1 

# Creating new column last_contact_date 
campaign["last_contact_date"] = pd.to_datetime("2022" + campaign["month"].astype(str) + \
                                               campaign["day"].astype(str), format='%Y%b%d') 
# Droping col month and day 
campaign.drop(["month","day"],axis = 1,inplace = True) 

# Save the DataFrames to csv file without an index
campaign.to_csv("campaign.csv" , index=False)

## step 3 Economics: splitting, Cleaning and Saving the data 

In [52]:
# all col we need 
economics = all_data[["client_id","emp_var_rate","cons_price_idx","euribor3m","nr_employed"]] 

# Rename columns 
economics.rename(columns = {'euribor3m' : 'euribor_three_months',
                            'nr_employed' : 'number_employed'
                           }, inplace=True)

# Save the DataFrames to csv file without an index
economics.to_csv("economics.csv", index=False)

## Checks that the data matches the requirements

In [53]:
print(client.columns)
print(campaign.columns)
print(economics.columns)

Index(['id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing',
       'loan'],
      dtype='object')
Index(['client_id', 'number_contacts', 'contact_duration', 'pdays',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'campaign_id', 'last_contact_date'],
      dtype='object')
Index(['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor_three_months',
       'number_employed'],
      dtype='object')


## step 4:

In [54]:
client_table = """CREATE TABLE client 
(
              id serial PRIMARY KEY,
              age INTEGER,
              job text,
              marital text,
              education text,
              credit_default boolean, 
              housing boolean,
              loan boolean
); 
\copy account from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [55]:
campaign_table = """CREATE TABLE campaign
(
              campaign_id serial PRIMARY KEY,
              client_id serial ,
              number_contacts integer,
              contact_duration integer,
              pdays integer,
              previous_campaign_contacts integer, 
              previous_outcome boolean,
              campaign_outcome boolean,
              last_contact_date date,
              foreign key(client_id) references client
); 
\copy account from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [56]:
economics_table = """CREATE TABLE economics 
(
              client_id serial PRIMARY KEY,
              emp_var_rate float,
              cons_price_idx float,
              euribor_three_months float,
              number_employed float,
              foreign key(client_id) references client
); 
\copy account from 'economics.csv' DELIMITER ',' CSV HEADER
"""